In [1]:
%%bash
chia version
cdv --version
python --version

1.6.1rc3.dev49
cdv, version 1.1.2
Python 3.10.8


In [2]:
# chia libraries
from blspy import (PrivateKey, AugSchemeMPL, G1Element, G2Element)

from chia.consensus.default_constants import DEFAULT_CONSTANTS
from chia.types.blockchain_format.coin import Coin
from chia.types.blockchain_format.program import Program
from chia.types.coin_spend import CoinSpend
from chia.types.condition_opcodes import ConditionOpcode
from chia.types.spend_bundle import SpendBundle
from chia.util.hash import std_hash
from chia.wallet.puzzles import p2_delegated_puzzle_or_hidden_puzzle

from chia.wallet.puzzles.p2_delegated_puzzle_or_hidden_puzzle import (
    DEFAULT_HIDDEN_PUZZLE_HASH,
    calculate_synthetic_secret_key,
    puzzle_for_pk,
    puzzle_for_conditions,
    solution_for_conditions,
)

from clvm.casts import int_to_bytes
from clvm_tools.clvmc import compile_clvm_text
from clvm_tools.binutils import disassemble

# utils & tic tac toe helper code
import sys
sys.path.insert(0, "../../../shared")
from utils import (load_program, print_program, print_puzzle, print_json, print_push_tx_result)


In [3]:
from chia.rpc.full_node_rpc_client import FullNodeRpcClient
from chia.util.config import load_config
from chia.util.default_root import DEFAULT_ROOT_PATH
from chia.util.ints import uint16

config = load_config(DEFAULT_ROOT_PATH, "config.yaml")
selected_network = config["selected_network"]
genesis_challenge = config["network_overrides"]["constants"][selected_network]["GENESIS_CHALLENGE"]

self_hostname = config["self_hostname"] # localhost
full_node_rpc_port = config["full_node"]["rpc_port"] # 8555


### Alice's
```sh
❯ chia keys derive -f $alice_fp wallet-address -i 0 --show-hd-path
Wallet address 0 (m/12381/8444/2/0): txch1n8s8aralslwl8ae2720rj30nt79qfspdfczxqreacjtyust8q3wsh8zj2d

~
❯ cdv decode txch1n8s8aralslwl8ae2720rj30nt79qfspdfczxqreacjtyust8q3wsh8zj2d
99e07e8fbf87ddf3f72af29e3945f35f8a04c02d4e04600f3dc4964e4167045d

~
❯ chia keys derive -f $alice_fp child-key -t wallet -i 0 --show-hd-path -s
Wallet public key 0 (m/12381/8444/2/0): 920c4790087dcf50e1b0cf565e58900b76743d2dd702c86a448f91acb3e8c0f7d66bf9ad637029bb319b60195b5bef72
Wallet private key 0 (m/12381/8444/2/0): 3c779a6deca5bce66a0738b957779d9eb355137041a3c902bee92d6ca3f396a4
```

In [4]:
alice_sk = PrivateKey.from_bytes(bytes.fromhex("3c779a6deca5bce66a0738b957779d9eb355137041a3c902bee92d6ca3f396a4"))
alice_pk = alice_sk.get_g1()
alice_ph = bytes.fromhex("99e07e8fbf87ddf3f72af29e3945f35f8a04c02d4e04600f3dc4964e4167045d")
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
coin_records = await full_node_client.get_coin_records_by_puzzle_hash(alice_ph, include_spent_coins = False)
full_node_client.close()
await full_node_client.await_closed()

alice_coin = coin_records[0].coin
print_json(alice_coin.to_json_dict())

{
    "amount": 10000000000000,
    "parent_coin_info": "0xd3f33700d7aa4515f5226e0283cc8426bee8ca2dfe4589cde5622b7d463cc88a",
    "puzzle_hash": "0x99e07e8fbf87ddf3f72af29e3945f35f8a04c02d4e04600f3dc4964e4167045d"
}


In [5]:
message = bytes("hello", "utf-8")
announcement = std_hash(message)
alice_conditions = [
    [
        ConditionOpcode.CREATE_COIN_ANNOUNCEMENT,
        announcement
    ],

    # return
    [
        ConditionOpcode.CREATE_COIN, 
        alice_coin.puzzle_hash, 
        alice_coin.amount
    ]
]

puzzle_reveal = puzzle_for_pk(alice_pk)
delegated_puzzle: Program = puzzle_for_conditions(alice_conditions) 
solution = solution_for_conditions(alice_conditions)

alice_coin_spend = CoinSpend(
    alice_coin,
    puzzle_reveal,
    solution
)
    
synthetic_sk: PrivateKey = calculate_synthetic_secret_key(
    alice_sk,
    DEFAULT_HIDDEN_PUZZLE_HASH
)

alice_sig = AugSchemeMPL.sign(synthetic_sk,
    (
        delegated_puzzle.get_tree_hash()
        + alice_coin.name()
        + bytes.fromhex(genesis_challenge)
    )
)

alice_spend_bundle = SpendBundle(
    [alice_coin_spend],
    alice_sig
)
print_json(alice_spend_bundle.to_json_dict())

{
    "aggregated_signature": "0x80d9598bacc28be2acb759bc76e29590ea3eb14ff415bb3df5cb0cd532728582bbeb691187e4bc1a9c4b86cfec8e809413c8da2409bea7f156446db7ed48ad6af7683cfe4afe9aafaa62f0df5d45e38c72780c4c7c012119409ef0ee9fad9553",
    "coin_solutions": [
        {
            "coin": {
                "amount": 10000000000000,
                "parent_coin_info": "0xd3f33700d7aa4515f5226e0283cc8426bee8ca2dfe4589cde5622b7d463cc88a",
                "puzzle_hash": "0x99e07e8fbf87ddf3f72af29e3945f35f8a04c02d4e04600f3dc4964e4167045d"
            },
            "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff0bffff01ff02ffff03ffff09ff05ffff1dff0bffff1effff0bff0bffff02ff06ffff04ff02ffff04ff17ff8080808080808080ffff01ff02ff17ff2f80ffff01ff088080ff0180ffff01ff04ffff04ff04ffff04ff05ffff04ffff02ff06ffff04ff02ffff04ff17ff80808080ff80808080ffff02ff17ff2f808080ff0180ffff04ffff01ff32ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff06ffff04ff02ffff04ff09ff80808080ffff02ff06ffff04ff02ffff04ff0dff8080808

In [8]:
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
result = await full_node_client.push_tx(alice_spend_bundle)
full_node_client.close()
await full_node_client.await_closed()
print(result)

{'status': 'SUCCESS', 'success': True}


### Bob's
```sh
❯ chia keys derive -f $bob_fp wallet-address -i 0 --show-hd-path
Wallet address 0 (m/12381/8444/2/0): txch1kn7ge94x4l3kcv74ywmm40d8grashjuszc4rx92t35swryhcypusgy53ls

~
❯ cdv decode txch1kn7ge94x4l3kcv74ywmm40d8grashjuszc4rx92t35swryhcypusgy53ls
b4fc8c96a6afe36c33d523b7babda740fb0bcb90162a33154b8d20e192f82079

~
❯ chia keys derive -f $bob_fp child-key -t wallet -i 0 --show-hd-path -s
Wallet public key 0 (m/12381/8444/2/0): a9d51a7afd3614725f44a1bd3720eea37cbf983b2335d90cf4674acf52085da15273b679cd05ac4f960272d23f5a4831
Wallet private key 0 (m/12381/8444/2/0): 1d9116c0fba5ec752d8dda784b9796fe3ae0cdb9d8daf8cca774c62c3c58c29c

```

In [6]:
bob_sk = PrivateKey.from_bytes(bytes.fromhex("1d9116c0fba5ec752d8dda784b9796fe3ae0cdb9d8daf8cca774c62c3c58c29c"))
bob_pk = bob_sk.get_g1()
bob_ph = bytes.fromhex("b4fc8c96a6afe36c33d523b7babda740fb0bcb90162a33154b8d20e192f82079")
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
coin_records = await full_node_client.get_coin_records_by_puzzle_hash(bob_ph, include_spent_coins = False)
full_node_client.close()
await full_node_client.await_closed()

bob_coin = coin_records[0].coin
print_json(bob_coin.to_json_dict())

{
    "amount": 1000000000000,
    "parent_coin_info": "0x4b36f6f9fec96fd81512486690d85716e702d50434ac75da4afb8805fd2e8828",
    "puzzle_hash": "0xb4fc8c96a6afe36c33d523b7babda740fb0bcb90162a33154b8d20e192f82079"
}


In [7]:
bob_conditions = [
    [
        ConditionOpcode.ASSERT_COIN_ANNOUNCEMENT,
        std_hash(alice_coin.name() + announcement)
    ],
    
    # return
    [
        ConditionOpcode.CREATE_COIN, 
        bob_coin.puzzle_hash, 
        bob_coin.amount
    ]
]

puzzle_reveal = puzzle_for_pk(bob_pk)
delegated_puzzle: Program = puzzle_for_conditions(bob_conditions) 
solution = solution_for_conditions(bob_conditions)

bob_coin_spend = CoinSpend(
    bob_coin,
    puzzle_reveal,
    solution
)
    
synthetic_sk: PrivateKey = calculate_synthetic_secret_key(
    bob_sk,
    DEFAULT_HIDDEN_PUZZLE_HASH
)

bob_sig = AugSchemeMPL.sign(synthetic_sk,
    (
        delegated_puzzle.get_tree_hash()
        + bob_coin.name()
        + bytes.fromhex(genesis_challenge)
    )
)

bob_spend_bundle = SpendBundle(
    [bob_coin_spend],
    bob_sig
)
print_json(bob_spend_bundle.to_json_dict())

{
    "aggregated_signature": "0xb7b151ad6edf35bf4f2de384cd740dff9d9f833bfe81c1f846f48007fe86c2dbc619bdd5499e0d19c5223f4b7bebc05a17312758bb82b6867c0a4ee4a554bef02f6446cc0d9f95c78856ff69be9266f88f8febf46d945838fe38d43b3ddc7717",
    "coin_solutions": [
        {
            "coin": {
                "amount": 1000000000000,
                "parent_coin_info": "0x4b36f6f9fec96fd81512486690d85716e702d50434ac75da4afb8805fd2e8828",
                "puzzle_hash": "0xb4fc8c96a6afe36c33d523b7babda740fb0bcb90162a33154b8d20e192f82079"
            },
            "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff0bffff01ff02ffff03ffff09ff05ffff1dff0bffff1effff0bff0bffff02ff06ffff04ff02ffff04ff17ff8080808080808080ffff01ff02ff17ff2f80ffff01ff088080ff0180ffff01ff04ffff04ff04ffff04ff05ffff04ffff02ff06ffff04ff02ffff04ff17ff80808080ff80808080ffff02ff17ff2f808080ff0180ffff04ffff01ff32ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff06ffff04ff02ffff04ff09ff80808080ffff02ff06ffff04ff02ffff04ff0dff80808080

In [9]:
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
result = await full_node_client.push_tx(bob_spend_bundle)
full_node_client.close()
await full_node_client.await_closed()
print(result)

ValueError: {'error': 'Failed to include transaction 2061901dc155228c89cdf0490ae8695caba6b86ee256fe0b079fcb99d645af53, error ASSERT_ANNOUNCE_CONSUMED_FAILED', 'success': False}

In [10]:
spend_bundle = SpendBundle(
    [alice_coin_spend, bob_coin_spend],
    AugSchemeMPL.aggregate([alice_sig, bob_sig])
)
print_json(spend_bundle.to_json_dict())

{
    "aggregated_signature": "0xab3b2fbcd163be413d69ea1f7dab816b50b0e5085f3f4536758b75a056fddd5d8ef39dc73916e08f46cf03b4269f65ae147d236720996201c6c8518bfb360e75b269f11a919d97132e4d1a161ed79ea01ee9434af45a6fefc76fe6120238577d",
    "coin_solutions": [
        {
            "coin": {
                "amount": 10000000000000,
                "parent_coin_info": "0x2844cd3578ea58014befe5311bbf74587da8663767bf7b037923de7ec294e82b",
                "puzzle_hash": "0x99e07e8fbf87ddf3f72af29e3945f35f8a04c02d4e04600f3dc4964e4167045d"
            },
            "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff0bffff01ff02ffff03ffff09ff05ffff1dff0bffff1effff0bff0bffff02ff06ffff04ff02ffff04ff17ff8080808080808080ffff01ff02ff17ff2f80ffff01ff088080ff0180ffff01ff04ffff04ff04ffff04ff05ffff04ffff02ff06ffff04ff02ffff04ff17ff80808080ff80808080ffff02ff17ff2f808080ff0180ffff04ffff01ff32ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff06ffff04ff02ffff04ff09ff80808080ffff02ff06ffff04ff02ffff04ff0dff8080808

In [11]:
full_node_client = await FullNodeRpcClient.create(
    self_hostname, uint16(full_node_rpc_port), DEFAULT_ROOT_PATH, config
)
result = await full_node_client.push_tx(spend_bundle)
full_node_client.close()
await full_node_client.await_closed()
print(result)

{'status': 'SUCCESS', 'success': True}
